In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from Utils.masking import MaskGenerator, visualise_mask
import matplotlib.pyplot as plt
import math

In [31]:
import numpy as np

data = torch.randn(100, 64, 1, 128, 128)
rotations = torch.randn(100, 64, 3)
B = 10
def get_batch(data, rotations, numpy=True):
    current_position = 0
    if numpy:
        rot_idxs = np.random.randint(0, 64, (B, 2))
    else:
        rot_idxs = torch.randint(0, 64, (B, 2))
    imgs1 = data[current_position:current_position+B, rot_idxs[:, 0], :, :, :]
    rots1 = rotations[current_position:current_position+B, rot_idxs[:, 0], :,]
    imgs2 = data[current_position:current_position+B, rot_idxs[:, 1], :, :, :]
    rots2 = rotations[current_position:current_position+B, rot_idxs[:, 1], :,]
    return imgs1, imgs2, rots1, rots2

# imgs1, imgs2, rots1, rots2 = get_batch(data, rotations)
# imgs1.shape, imgs2.shape, rots1.shape, rots2.shape
%timeit get_batch(data, rotations, numpy=True)
%timeit get_batch(data, rotations, numpy=False)

268 µs ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
242 µs ± 10 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
path = 'configs/test.yaml'
import yaml

with open(path, 'r') as file:
    config = yaml.safe_load(file)


In [22]:
from Methods.BYOL.model import BYOL
model = BYOL(1, 'resnet18', 128)
teacher = model.copy()
for name, param in teacher.encoder.transformer_pool.named_parameters():
    print(name, param.shape)

pos_embed torch.Size([1, 64, 512])
cls_token torch.Size([1, 1, 512])
registers torch.Size([1, 4, 512])
layers.encoder_layer_0.ln_1.weight torch.Size([512])
layers.encoder_layer_0.ln_1.bias torch.Size([512])
layers.encoder_layer_0.self_attention.in_proj_weight torch.Size([1536, 512])
layers.encoder_layer_0.self_attention.in_proj_bias torch.Size([1536])
layers.encoder_layer_0.self_attention.out_proj.weight torch.Size([512, 512])
layers.encoder_layer_0.self_attention.out_proj.bias torch.Size([512])
layers.encoder_layer_0.ln_2.weight torch.Size([512])
layers.encoder_layer_0.ln_2.bias torch.Size([512])
layers.encoder_layer_0.mlp.0.weight torch.Size([1024, 512])
layers.encoder_layer_0.mlp.0.bias torch.Size([1024])
layers.encoder_layer_0.mlp.3.weight torch.Size([512, 1024])
layers.encoder_layer_0.mlp.3.bias torch.Size([512])
ln.weight torch.Size([512])
ln.bias torch.Size([512])


In [13]:
filepath = 'configs/test'
if not filepath.endswith('.yaml'):
    filepath += '.yaml'
with open(filepath, 'r') as file:
    yaml_cfgs = yaml.safe_load(file)

cfgs = []
for yaml_cfg in yaml_cfgs['cfgs']:
    print(yaml_cfg)
    if yaml_cfg['dataset'] == 'modelnet10':
        cfgs.append(modelnet10_cfg(**yaml_cfg))
    elif yaml_cfg['dataset'] == 'mnist':
        cfgs.append(mnist_cfg(**yaml_cfg))
    else:
        raise ValueError(f'Dataset {yaml_cfg["dataset"]} not supported')

{'dataset': 'mnist', 'experiment': 'testing', 'trial': 'AE', 'model_type': 'AE', 'save': True, 'num_epochs': 2, 'warmup': 0}


NameError: name 'mnist_cfg' is not defined

In [10]:
config['cfgs'][0]

{'experiment': 'testing',
 'trial': 'AE',
 'model_type': 'AE',
 'save': True,
 'num_epochs': 2,
 'warmup': 0}